### Loading the data and making the training and validation sets 

In [ ]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2021-06-05 19:46:54--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-06-05 19:46:54 (135 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
# The dataset is Natural Language Processing with Disaster Tweets from Kaggle.
# If the above link is not available download zip from kaggle.

In [ ]:
!unzip "/content/nlp_getting_started.zip"

Archive:  /content/nlp_getting_started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("/content/train.csv")

test_df = pd.read_csv("/content/test.csv")

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
print(train_df["text"][45])
print(train_df["target"][45])

I gained 3 followers in the last week. You? Know your stats and grow with http://t.co/TIyUliF5c6
0


In [ ]:
print(train_df["text"][50])
print(train_df["target"][50])

Deputies: Man shot before Brighton home set ablaze http://t.co/gWNRhMSO8k
1


In [ ]:
# we can see that 1 is for disaster and 0 is for not disaster.

In [ ]:
# Lets shuffle the data as we can see the df has a lot of 1s in the beginning.

In [ ]:
train_df_shuffled = train_df.sample(frac = 1,random_state= 42)

In [ ]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# Seeing how many disaster tweets and how many non-disaster tweets are there in the dataset

train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# 0    4342
# 1    3271

# We can see we have more non-disaster tweets

In [ ]:
len(train_df),len(test_df)

(7613, 3263)

In [ ]:
# Seeing some random tweets

import random

for i in range(5):

  random_number = random.randint(0,len(train_df)-1)
  tar = train_df["target"][random_number]

  print("---------------------------")

  if tar == 1:
    print("Disaster")
  else:
    print("Not a disaster")

  print("---------------------------")

  print(train_df["text"][random_number])

  print("\n\n")

---------------------------
Not a disaster
---------------------------
I added a video to a @YouTube playlist http://t.co/aTCMrjzJTp Nicki Minaj - Up In Flames (Official Video)



---------------------------
Not a disaster
---------------------------
@kinkyconnors IM sorry for my meltdown last night lmao but I'm getting my tooth fixed Friday ??????????????????????????????????



---------------------------
Disaster
---------------------------
Madhya Pradesh Train Derailment: Village Youth Saved Many Lives



---------------------------
Disaster
---------------------------
Now that's what you call a batting collapse #theashes



---------------------------
Disaster
---------------------------
'Money can't buy happiness' is just a lie we tell poor people to keep them from rioting.





In [ ]:
# Creating the training and validation sets

from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                         train_df_shuffled["target"].to_numpy(),
                                                                         test_size  = 0.1,
                                                                         random_state = 42)

In [ ]:
len(train_sentences),len(val_sentences)

(6851, 762)

In [ ]:
print(train_sentences[:5])
print(train_labels[:5])

['@mogacola @zamtriossu i screamed after hitting tweet'
 'Imagine getting flattened by Kurt Zouma'
 '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....'
 "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet"
 'Somehow find you and I collide http://t.co/Ee8RpOahPk']
[0 0 1 0 0]


In [ ]:
import tensorflow as tf

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
# JUST AN EXAMPLE OF TEXT VECTORIZER. MADE AGAIN LATER WITH RIGHT VLAUES

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens=True)

In [ ]:
# Set max vocab length 

max_vocab_length = 10000

# Find average length to set it as the max length 

max_length = round(sum(len(i.split()) for i in train_sentences)/len(train_sentences))

print(max_length)


15


In [ ]:
# Making the text vectorizer

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [ ]:
# Fit/Adapt the vectorizer

text_vectorizer.adapt(train_sentences)

In [ ]:
# Using the text_vectorizer on a random sentence

# The text_vectorizers needs a list as input

sample_sentence = "The cyclone is heading towards Orissa."

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2,  582,    9, 2964, 3316,    1,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
# We can see that the shape is 1,15  as the max length is set to 15

# Since the sentence has only 7 words thus all other are 0.

In [ ]:
# Seeing the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()

words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'as',
 'up',
 'so',
 'just',
 'but',
 'me',
 'im',
 'your',
 'not',
 'amp',
 'out',
 'its',
 'will',
 'an',
 'no',
 'has',
 'fire',
 'after',
 'all',
 'when',
 'we',
 'if',
 'now',
 'via',
 'new',
 'more',
 'get',
 'or',
 'about',
 'what',
 'he',
 'people',
 'news',
 'been',
 'over',
 'one',
 'how',
 'dont',
 'they',
 'who',
 'into',
 'were',
 'do',
 'us',
 '2',
 'can',
 'video',
 'emergency',
 'there',
 'disaster',
 'than',
 'police',
 'would',
 'his',
 'still',
 'her',
 'some',
 'body',
 'storm',
 'crash',
 'burning',
 'suicide',
 'back',
 'man',
 'california',
 'why',
 'time',
 'them',
 'had',
 'buildings',
 'rt',
 'first',
 'cant',
 'see',
 'got',
 'day',
 'off',
 'our',
 'going',
 'nuclear',
 'know',
 'world',
 'bomb',
 'fires',
 'love',
 'killed',
 'go',
 'attack',
 'youtube',
 'dead

In [ ]:
# number of words in vocalbulary

len(words_in_vocab)

10000

In [ ]:
# It was expected to be 10000 as we set the max size of vocabulary as well.

In [ ]:
# Top 5 words and last 5v words

print(words_in_vocab[:5])

print()

print(words_in_vocab[-5:])

['', '[UNK]', 'the', 'a', 'in']

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
# We can see the most common words are spaces (" "), then [UNK] which were the punctuations,etc. Then "the","a,"in" and other such words.

# The least common words are pages, paeds, pads,padres,etc

##### Embedding

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                      output_dim = 128,
                                      input_length = max_length)

In [ ]:
embedding

In [ ]:
# Trying embedding on a random sentence.

# the embedding requires a text_vectorizerized input


sample_embed = embedding(text_vectorizer([sample_sentence]))

sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 2.61947028e-02, -4.09372151e-05, -4.58424352e-02, ...,
         -3.21570039e-02,  1.31714009e-02,  3.21646817e-02],
        [ 8.75718519e-03,  4.92960475e-02, -2.43463404e-02, ...,
         -1.01518035e-02, -6.99303299e-03, -2.37602834e-02],
        [ 1.18264928e-02, -3.37104350e-02, -1.48755796e-02, ...,
         -2.82621384e-02, -1.43779144e-02,  3.82083096e-02],
        ...,
        [-1.52111426e-02, -3.78493182e-02, -3.44001427e-02, ...,
          1.30989589e-02,  8.08141381e-03, -1.84226148e-02],
        [-1.52111426e-02, -3.78493182e-02, -3.44001427e-02, ...,
          1.30989589e-02,  8.08141381e-03, -1.84226148e-02],
        [-1.52111426e-02, -3.78493182e-02, -3.44001427e-02, ...,
          1.30989589e-02,  8.08141381e-03, -1.84226148e-02]]],
      dtype=float32)>

In [ ]:
# Each int is described by 128 float numbers. This is done for each of the 15 int which were vectorized from a sentence.

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.01521114, -0.03784932, -0.03440014,  0.0467254 ,  0.02862969,
       -0.01942397,  0.04052866,  0.04183784, -0.04519415,  0.01066961,
        0.00096393,  0.04858148,  0.03935734, -0.03031158, -0.02807545,
       -0.03609843, -0.00242481, -0.01969994,  0.04193461,  0.04157324,
       -0.01551242,  0.02553216, -0.00747577, -0.00166125,  0.00314926,
       -0.04937055,  0.03339041, -0.03579167,  0.03000417,  0.04695631,
        0.04686364, -0.01749649,  0.03107666,  0.0028071 ,  0.03879431,
       -0.02434716, -0.00592845,  0.01457116, -0.02398125,  0.02512806,
       -0.00508202, -0.02397947, -0.04136392, -0.04320109,  0.02137304,
        0.01202179,  0.02310315,  0.00550558, -0.02458713,  0.03099999,
        0.04625131,  0.01729042,  0.02330125,  0.00631893,  0.02282475,
        0.00988812, -0.04046027, -0.04254219, -0.04903687,  0.0388085 ,
        0.00454953,  0.00651996,  0.00808054, -0.02310002, -0.02422491,
       -0.017536